In [2]:
import gmplot
import pandas as pd
%pylab inline
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.preprocessing import Imputer
from __future__ import division

def get_dataframes(zone):
    dflist = []

    df_prob = pd.read_csv('/home/datascience/labs/lab10/cleaning2/'+str(zone)+'/prob.tsv', sep='\t', lineterminator='\n',
                     na_values=['-'])

    df_flow = pd.read_csv('/home/datascience/labs/lab10/cleaning2/'+str(zone)+'/flow.tsv', sep='\t', lineterminator='\n',
                     na_values=['-'])
    prob_columns = []
    flow_columns = []
    for i in range(1,len(df_prob.columns)+1):
        prob_columns.append('prob'+str(i))
        flow_columns.append('flow'+str(i))
    df_prob.columns = prob_columns 
    df_flow.columns = flow_columns 
    dflist.append(df_prob)    
    dflist.append(df_flow)           
    return dflist,flow_columns,prob_columns

def confidence(df,probs_x,coefs_x):
    df['prob_sum']=0.0
    biglist = []
    coeff_sum=0.0
    for i in range(len(probs_x)):
        innerlist = []
        coeff = coefs_x[i]
        prob = probs_x[i]
        coeff_sum += coeff
        #print(prob_sum)
        df['prob_sum'] +=  coeff*df[prob]
        #biglist.append(innerlist)
    
    #prob_sum = map(sum, zip(*biglist))
    confidence = df['prob_sum']/coeff_sum
    return confidence

def getmethod1_df(dflist,flows,probs):
    df = pd.concat(dflist, axis=1)
    length =len(flows)
    
    if length<=1:
        return df
    else:
        condition0 = (df.isnull().any(axis=1))    
        cond = (~condition0)

        for i in range(0,length):
            prob = probs[i]
            cond = (cond) & (df[prob]!=0)

        df_nan = df[condition0]    
        df_predict = df.copy()
        df_training = df[cond]

        for j in range(0,length):

            flows_x = flows[:]
            flow_to_predict = flows_x.pop(j)

            probs_x = probs[:]
            prob_to_predict = probs_x.pop(j)

            X = df_training.as_matrix(columns = flows_x)

            X_test =  df.as_matrix(columns = flows_x)

            y=df_training.as_matrix(columns = [flow_to_predict])

            regr = linear_model.LinearRegression()
            regr.fit(X,y)
            coefs = regr.coef_[0].tolist()
            imp = Imputer(missing_values='NaN', strategy='mean', axis=0)   
            df_predict[flow_to_predict]=regr.predict(imp.fit_transform(X_test))
            df_predict[prob_to_predict] = confidence(df,probs_x,coefs)


        for row in df_nan.itertuples():
            if np.isnan(row).any():
                for flow,prob in zip(flows,probs):
                    df_predict[flow][row[0]] = df[flow][row[0]]
                    df_predict[prob][row[0]] = df[prob][row[0]]

        return df_predict


    

Populating the interactive namespace from numpy and matplotlib


In [1]:
# method2 

def getmethod2_df(dflist,flows,probs):
    df = pd.DataFrame()    
    df = pd.concat(dflist, axis=1)
    print df.head(10)
    length = len(flows)
    df_list = []   
    
    for j in range(0,length):
        prob = probs[j]
        flow = flows[j]
        df_temp = df[[flow,prob]]
        c2 = df[prob].shift(-1)
        c1 = df[prob].shift(1)
        df_temp['c1+c2'] =c1+c2
        f1 = df_temp[flow].shift(1)
        f2 = df_temp[flow].shift(-1)

        w1 = c1/(c1+c2)
        w2 =1-w1

        series1 = w1*f1+w2*f2
        series2 =  pd.concat([c1, c2], axis=1).min(axis=1)

        df_temp['predict_flow'] = series1
        df_temp['predict_prob'] = series2

        cond = (df_temp['predict_flow'].isnull())

        df_temp['predict_flow'] = np.where(cond, df_temp[flow], df_temp['predict_flow'])    
        df_temp['predict_prob'] = np.where(cond, df_temp[prob], df_temp['predict_prob'])
                    
        final = pd.DataFrame()
        final[flow] = df_temp['predict_flow']
        final[prob] = df_temp['predict_prob']
        df_list.append(final)
    
    df_result = pd.concat(df_list, axis=1)    
    return df_result




In [3]:
# method 3 dataframe creation

def getmethod3_df(dflist):
    df3 = pd.concat([dflist[0], dflist[1]], axis=1)
    return df3

In [5]:
#merging all three methods

def merge_dataframes(df1,df2,df3,flows,probs):
    length = len(flows)
    merged_df = df1[flows]
    for j in range(0,length):       
        # get jth column from all
        #the three dfs
        
        predf = flows[j]
        conf = probs[j]
        

        pf1 = df1[predf]
        pf2 = df2[predf]
        pf3 = df3[predf]
        
        c1 = df1[conf]
        c2 = df2[conf]
        c3 = df3[conf]
        merged_df['c1+c2+c3'] = c1+c2+c3
        
        w1 = c1/(c1+c2+c3)
        w2 = c2/(c1+c2+c3)
        w3 = 1-w1-w2
        
        series = w1*pf1+w2*pf2+w3*pf3
        merged_df[predf] = series
        
        cond = (merged_df['c1+c2+c3']==0.0)
        merged_df[predf] = np.where(cond, df3[predf], merged_df[predf])
        merged_df.drop('c1+c2+c3', axis=1, inplace=True)

    return merged_df

In [10]:
#input_path = sys.argv[1]

def getflowcsv(zone):
    #zone = input_path.rsplit('/',1)[1]
    filename = str(zone)+'.flow.txt'
    
    frames_list,flow_cols,prob_cols = get_dataframes(zone)
    df1 = getmethod1_df(frames_list,flow_cols,prob_cols)
    df2 = getmethod2_df(frames_list,flow_cols,prob_cols)
    df3 = getmethod3_df(frames_list)
    
    merged_df = merge_dataframes(df1,df2,df3,flow_cols,prob_cols)
    merged_df = np.round(merged_df)
    print merged_df.head(10)
    for flow in flow_cols:
        merged_df[flow] = merged_df[flow].astype(int)
    merged_df.to_csv(filename, header=None, index=None,sep='\t')


In [11]:
getflowcsv(1160)

      prob1     prob2  prob3  flow1  flow2  flow3
0  0.000223  0.000249      0      9      4    255
1  0.000839  0.000427      0     11     10    255
2  0.000000  0.001775      0     -8      3    255
3  0.000676  0.000213      0      5      9    255
4  0.000506  0.000776      0      7     10    255
5  0.000709  0.000161      0      6      6    255
6  0.009856  0.000320      0      0      3    255
7  0.000543  0.000005      0     10      8    255
8  0.000003  0.000464      0      7      3    255
9  0.000127  0.000123      0     10      7    255


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame


   flow1  flow2  flow3
0     10      5      8
1     14      8      8
2     11      4      5
3      8      5      7
4     11      8      8
5      5      6      8
6      1     -0      7
7     10      6      8
8     11      3      8
9     13      7      8


In [20]:
%reset

,flow1,flow2,flow3,flow4
0,NaN,NaN,NaN,NaN
1,30.107215,22.208949,27.243324,30.701547
2,25.996994,14.825068,23.230338,24.628027
3,30.257752,16.641478,25.615272,29.865695
4,29.979510,15.258494,27.026484,30.962886
5,28.575593,16.108044,23.795718,31.196165
6,29.664154,14.417547,29.545023,31.738700
7,31.945007,16.637773,28.366178,31.587330
8,30.906862,18.835363,26.701383,34.851395
9,30.801765,17.479617,25.627818,33.163370


In [10]:
dfx = pd.read_csv('/home/datascience/labs/lab10/1160.flow.txt', sep='\t', lineterminator='\n', na_values=['-'])

In [11]:
condition0 = (dfx.isnull().any(axis=1))    
len(dfx[condition0])

0

In [11]:
  df_flow = pd.read_csv('/home/datascience/labs/lab10/cleaning2/1160/flow.tsv', sep='\t', lineterminator='\n',
                     na_values=['-'])

In [13]:
condition1 = (df_flow.isnull().any(axis=1))    
len(df_flow[condition1])

0